In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import mediapipe as mp

# Paths
open_dir = "G:/zolo/classroom-attention-monitor/datasets/eyes/Open_Eyes"
closed_dir = "G:/zolo/classroom-attention-monitor/datasets/eyes/Closed_Eyes"
output_csv = "G:/zolo/classroom-attention-monitor/datasets/eyes/eye_landmarks.csv"

# MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)

# Eye landmark indexes (MediaPipe face mesh)
LEFT_EYE = [33, 133, 160, 159, 158, 144, 153, 154, 155]
RIGHT_EYE = [362, 263, 387, 386, 385, 373, 380, 381, 382]

data = []

def process_folder(folder_path, label):
    for file in tqdm(os.listdir(folder_path)):
        if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        img_path = os.path.join(folder_path, file)
        img = cv2.imread(img_path)
        if img is None:
            continue

        img = cv2.resize(img, (256, 256))  # Add this line
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(img_rgb)

        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0].landmark

            feature_vector = []
            for idx in LEFT_EYE + RIGHT_EYE:
                lm = landmarks[idx]
                feature_vector.extend([lm.x, lm.y])

            feature_vector.append(label)
            data.append(feature_vector)
        if results.multi_face_landmarks:
            print(f"✅ Landmarks found in: {file}")
        else:
            print(f"❌ No face detected in: {file}")



# Process open eyes
process_folder(open_dir, 0)

# Process closed eyes
process_folder(closed_dir, 1)

# Save to CSV
columns = [f"{eye}{i}_{axis}" for eye in ['L', 'R'] for i in range(9) for axis in ['x', 'y']]
columns.append("label")
df = pd.DataFrame(data, columns=columns)
df.to_csv(output_csv, index=False)
print("✅ Landmark data saved:", output_csv)
df.head()


In [16]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Paths
data_dir = "G:/zolo/classroom-attention-monitor/datasets/eyes"
model_save_path = "G:/zolo/classroom-attention-monitor/models/eye_state_cnn.h5"

# Params
IMG_SIZE = 34
categories = ['Open_Eyes', 'Closed_Eyes']
label_map = {'Open_Eyes': 0, 'Closed_Eyes': 1}

# Load Data
X = []
y = []

for label in categories:
    folder = os.path.join(data_dir, label)
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            X.append(img)
            y.append(label_map[label])

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0
y = to_categorical(y, num_classes=2)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# Save model
model.save(model_save_path)
print(f"✅ Model saved to: {model_save_path}")


c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 420,610 (1.60 MB)

 Trainable params: 420,610 (1.60 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7749 - loss: 0.4544 - val_accuracy: 0.9725 - val_loss: 0.0905
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9787 - loss: 0.0705 - val_accuracy: 0.9800 - val_loss: 0.0584
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9833 - loss: 0.0484 - val_accuracy: 0.9837 - val_loss: 0.0472
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.9926 - loss: 0.0264 - val_accuracy: 0.9962 - val_loss: 0.0195
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9936 - loss: 0.0224 - val_accuracy: 0.9912 - val_loss: 0.0229
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9920 - loss: 0.0236 - val_accuracy: 0.9937 - val_loss: 0.0165
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.9966 - loss: 0.0085 - val_accuracy: 0.9937 - val_loss: 0.0119
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9988 - loss: 0.0048 - val_accu

✅ Model saved to: G:/zolo/classroom-attention-monitor/models/eye_state_cnn.h5
